In [ ]:
!pip install timm

In [32]:
import os
import zipfile
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from google.colab import drive
from PIL import Image
from sklearn.metrics import (classification_report, confusion_matrix, log_loss,
                             roc_auc_score)
from torch.utils.data import DataLoader, Dataset
from torchvision.models import ResNeXt50_32X4D_Weights, resnext50_32x4d
from tqdm import tqdm as tqdm

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
with zipfile.ZipFile('/content/drive/MyDrive/UCCD3074/Asm2/cassava-leaf-disease-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [12]:
############################
#Coded by Leong Wai Yin
############################
df = pd.read_csv('/content/cassava-leaf-disease-classification/train.csv')

#train, val, test split (70:15:15)
df_train = df.sample(frac=0.7, random_state=3074)
val_test = df.loc[~df.index.isin(df_train.index)]
df_test = val_test.sample(frac=0.5, random_state=3074)
df_valid = val_test.loc[~val_test.index.isin(df_test.index)]
print(len(df))
print(len(df_train))
print(len(df_valid))
print(len(df_test))

21397
14978
3209
3210


In [13]:
############################
#Coded by Leong Wai Yin
############################
def get_transform(mode=0):

  train_transform = [
      transforms.Compose([    #no augmentation
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([    #slight augmentation with cropping
        transforms.Resize(512),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([    #heavy augmentation
        transforms.RandomResizedCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=(0,180)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
      ])
  ]

  val_transform = [
      transforms.Compose([
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
      ])
  ]  

  return train_transform[mode], val_transform[mode]

In [14]:
def get_testloader(batch_size=32, val_transform=None):

  testset = CasavaDataset(df_test, '/content/cassava-leaf-disease-classification/train_images', transform=val_transform)
  testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

  return testloader

In [15]:
############################
#Adapted from lab6A
############################
def build_network(weights=None, model):
  if model == 'resnext50':
    net = resnext50_32x4d(weights=weights)
    in_c = net.fc.in_features    #get fc input shape
    net.fc = nn.Linear(in_c, 5)   #match model output with number of labels

  if model == 'efficientnetb0':
    net = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
    in_c = net.classifier.in_features
    net.classifier = nn.Linear(in_c, 5)
  return net

In [16]:
############################
#Adapted from Lab6B
############################
class CasavaDataset(Dataset):

    def __init__(self, csv, root, transform=None):
        self.csv = csv    #store dataset partition csv
        self.root = root    #image dataset directory
        self.transform = transform    #preprocessing and augmentation method
        self.classes = ['CBB', 'CBSD', 'CGM', 'CMD', 'H']   #list of classes

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        
        #get image by index
        row=self.csv.iloc[idx]
        img = os.path.join(self.root, row.image_id)
        image = Image.open(img)
        
        #transformation
        if self.transform is not None:
          image = self.transform(image)
        
        #get image label
        label = row.label
        
        return image, label

In [21]:
############################
#Coded by Leong Wai Yin
############################
def inference(model, testloader):
    model.eval()
    LOGITS=[]

    #inference by batch
    for inputs, targets in tqdm(testloader):
        
        #use gpu if available
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        #zero gradient
        with torch.no_grad():
            outputs = model(inputs)

            #append prediction outputs together
            LOGITS.append(outputs.detach().cpu())

    #normalize and flatten prediction probabilities
    PROBS = torch.sigmoid(torch.cat(LOGITS)).numpy().squeeze()

    return PROBS

In [22]:
def resnext50_inference():
    net = build_network('IMAGENET1K_V2', model='resnext50')
    state_dict = torch.load('/content/drive/MyDrive/casava/resnext50/resnext50_20_best.pth')
    net.load_state_dict(state_dict, strict=True)

    _, test_transform = get_transform(mode=2)

    testloader = get_testloader(64, test_transform)

    resnext50_probs = inference(net.cuda(), testloader)

    return resnext50_probs

100%|██████████| 51/51 [01:18<00:00,  1.54s/it]


In [ ]:
def efficientnetb0_inference():
    net = build_network(model='efficientnetb0')
    state_dict = torch.load('/content/drive/MyDrive/casava/resnext50/resnext50_20_best.pth')
    net.load_state_dict(state_dict, strict=True)

    _, test_transform = get_transform(mode=2)

    testloader = get_testloader(64, test_transform)

    efficientnetb0_probs = inference(net.cuda(), testloader)

    return efficientnetb0_probs

In [30]:
vit_probs = pd.read_csv('/content/drive/MyDrive/vit_probs.csv').to_numpy(dtype='float32')
vit_probs

array([[0.24364243, 0.15507847, 0.15002882, 0.6959583 , 0.9890131 ],
       [0.02843603, 0.02479816, 0.31237   , 0.9905874 , 0.32167307],
       [0.6959583 , 0.05749328, 0.19559409, 0.23231015, 0.9046505 ],
       ...,
       [0.03676946, 0.01542455, 0.15713686, 0.9879462 , 0.7879312 ],
       [0.13386749, 0.20307462, 0.02887091, 0.99444515, 0.12678517],
       [0.6522414 , 0.8056322 , 0.35577488, 0.28378138, 0.64332926]],
      dtype=float32)

In [31]:
#ensemble
ensemble_probs = (resnext50_probs + vit_probs)/2
ensemble_probs

array([[0.19373235, 0.11422366, 0.1369535 , 0.6271054 , 0.95016   ],
       [0.01525741, 0.01543134, 0.24563703, 0.99266934, 0.3495602 ],
       [0.6122197 , 0.11750431, 0.2012373 , 0.17020166, 0.87165344],
       ...,
       [0.01937464, 0.01052804, 0.1197926 , 0.9920248 , 0.6220048 ],
       [0.08026983, 0.11273554, 0.12222005, 0.9898571 , 0.09155048],
       [0.78178453, 0.57141435, 0.21767052, 0.1742041 , 0.573867  ]],
      dtype=float32)

In [33]:
############################
#Coded by Leong Wai Yin
############################
def evaluate(net_pred):
  actual = df_test.drop('image_id', axis=1).astype(str)
  pred = pd.DataFrame(np.argmax(net_pred, axis=1)).astype(str)
  cm = confusion_matrix(actual, pred)         #multi class confusion matrix
  cm_df = pd.DataFrame(cm,
                     index = ['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy'], 
                     columns = ['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy'])
  print(cm_df)
  print()
  cr = classification_report(actual, pred)    #multiclass classification report
  print(cr)
  print()


In [36]:
evaluate(resnext50_probs)

         CBB  CBSD  CGM   CMD  Healthy
CBB       85    10    5     9       60
CBSD       7   252   12    17       44
CGM        0     9  291    42       25
CMD        1    10   21  1909       19
Healthy   11    17   24    29      301

              precision    recall  f1-score   support

           0       0.82      0.50      0.62       169
           1       0.85      0.76      0.80       332
           2       0.82      0.79      0.81       367
           3       0.95      0.97      0.96      1960
           4       0.67      0.79      0.72       382

    accuracy                           0.88      3210
   macro avg       0.82      0.76      0.78      3210
weighted avg       0.89      0.88      0.88      3210




In [34]:
evaluate(vit_probs)

         CBB  CBSD  CGM   CMD  Healthy
CBB       89    11    7     9       53
CBSD      16   224   24    24       44
CGM        4    10  253    59       41
CMD        7     9   27  1882       35
Healthy   25    18   22    31      286

              precision    recall  f1-score   support

           0       0.63      0.53      0.57       169
           1       0.82      0.67      0.74       332
           2       0.76      0.69      0.72       367
           3       0.94      0.96      0.95      1960
           4       0.62      0.75      0.68       382

    accuracy                           0.85      3210
   macro avg       0.76      0.72      0.73      3210
weighted avg       0.85      0.85      0.85      3210




In [35]:
evaluate(ensemble_probs)

         CBB  CBSD  CGM   CMD  Healthy
CBB       89     8    6    10       56
CBSD      10   250   15    18       39
CGM        0     8  285    44       30
CMD        5     5   18  1911       21
Healthy   13    19   18    27      305

              precision    recall  f1-score   support

           0       0.76      0.53      0.62       169
           1       0.86      0.75      0.80       332
           2       0.83      0.78      0.80       367
           3       0.95      0.97      0.96      1960
           4       0.68      0.80      0.73       382

    accuracy                           0.88      3210
   macro avg       0.82      0.77      0.79      3210
weighted avg       0.89      0.88      0.88      3210


